In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
import os
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from tabulate import tabulate
from itertools import islice
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
import pickle
import glob
import random
import dask.dataframe as dd

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
data_path = '/content/drive/MyDrive/MovePort/'
#Modified_Df_PATH = '/content/drive/MyDrive/MovePortModif/'
participants = sorted([p for p in os.listdir(data_path) if p.isdigit()], key=int)
print("Participants disponibles :", participants)
activities = ['back', 'forward', 'halfsquat', 'still']
modalities = ['ips_1', 'emg_1', 'imu_1']

Participants disponibles : ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']


In [4]:
def list_missing_files(base_path, participants, activities, modalities):

  missing_files = {}
  for participant in participants:
      participant_missing = {}
      for activity in activities:
          for modality in modalities:
              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
              if not os.path.exists(file_path):
                  if activity not in participant_missing:
                      participant_missing[activity] = []
                  participant_missing[activity].append(modality)
      if participant_missing:
          missing_files[participant] = participant_missing

  return missing_files

In [5]:
# Lister les fichiers manquants
missing_files = list_missing_files(data_path, participants, activities, modalities)
# Afficher les résultats
participants_manque_files = []
print("Participants avec des fichiers de modalités manquants :")
for participant, activities in missing_files.items():
    print(f"Participant {participant}:")
    participants_manque_files.append(participant)
    for activity, modalities in activities.items():
        print(f"  Activité {activity}: Modalités manquantes: {', '.join(modalities)}")

Participants avec des fichiers de modalités manquants :
Participant 18:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 19:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 20:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 21:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 22:
  Activité back: Modalités manquantes: ips_1, emg_1, imu_1
Participant 23:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
  Activité halfsquat: Modalités manquantes: ips_1, emg_1, imu_1
Participant 24:
  Activité halfsquat: Modalités manquantes: ips_1, emg_1, imu_1


In [ ]:
#participants_manque_files

In [6]:
"""def get_column_structure_and_shape(base_path, participants, activities, modalities):

  column_structure = {}
  data_shapes = {}
  for modality in modalities:
      for participant in participants:
          for activity in activities:
              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
              if os.path.exists(file_path):
                  df = pd.read_csv(file_path, header=0)
                  column_structure[modality] = df.columns
                  data_shapes[modality] = df.shape
                  break
          if modality in column_structure:
              break
      if modality not in column_structure:
          raise ValueError(f"Structure des colonnes introuvable pour la modalité {modality}")

  return column_structure, data_shapes"""

'def get_column_structure_and_shape(base_path, participants, activities, modalities):\n\n  column_structure = {}\n  data_shapes = {}\n  for modality in modalities:\n      for participant in participants:\n          for activity in activities:\n              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")\n              if os.path.exists(file_path):\n                  df = pd.read_csv(file_path, header=0)\n                  column_structure[modality] = df.columns\n                  data_shapes[modality] = df.shape\n                  break\n          if modality in column_structure:\n              break\n      if modality not in column_structure:\n          raise ValueError(f"Structure des colonnes introuvable pour la modalité {modality}")\n\n  return column_structure, data_shapes'

In [7]:
"""def calculate_mean_structure(base_path, participants, activities, modalities):

  mean_data = {modality: [] for modality in modalities}
  for activity in activities:
      for modality in modalities:
          participant_pairs = random.sample(participants, 2)
          modality_data = []
          for participant in participant_pairs:
              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
              if os.path.exists(file_path):
                  try:
                      df = pd.read_csv(file_path, header=0)
                      numeric_df = df.apply(pd.to_numeric, errors='coerce')  # Convertir en numérique
                      modality_data.append(numeric_df)
                  except Exception as e:
                      print(f"Erreur lors du chargement de {file_path}: {e}")
          if modality_data:
              combined_df = pd.concat(modality_data)
              combined_df = combined_df.apply(pd.to_numeric, errors='coerce')
              combined_df = combined_df.dropna(axis=1, how='all')  # Supprimer les colonnes entièrement non numériques
              mean_df = combined_df.mean(axis=0).to_frame().transpose()  # Calculer uniquement les colonnes numériques
              mean_df = round(mean_df, 3)
              mean_data[modality].append(mean_df)

  for modality in modalities:
      mean_data[modality] = pd.concat(mean_data[modality], axis=0).mean(axis=0).to_frame().transpose()

  return mean_data"""

'def calculate_mean_structure(base_path, participants, activities, modalities):\n\n  mean_data = {modality: [] for modality in modalities}\n  for activity in activities:\n      for modality in modalities:\n          participant_pairs = random.sample(participants, 2)\n          modality_data = []\n          for participant in participant_pairs:\n              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")\n              if os.path.exists(file_path):\n                  try:\n                      df = pd.read_csv(file_path, header=0)\n                      numeric_df = df.apply(pd.to_numeric, errors=\'coerce\')  # Convertir en numérique\n                      modality_data.append(numeric_df)\n                  except Exception as e:\n                      print(f"Erreur lors du chargement de {file_path}: {e}")\n          if modality_data:\n              combined_df = pd.concat(modality_data)\n              combined_df = combined_df.apply(pd.to_numeric,

In [ ]:
def create_missing_files_with_mean(base_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes):

  participant_path = os.path.join(base_path, str(participant_id))
  for activity in activities:
      activity_path = os.path.join(participant_path, activity)
      if not os.path.exists(activity_path):
          os.makedirs(activity_path)  # Crée le dossier d'activité s'il n'existe pas
      for modality in modalities:
          file_path = os.path.join(activity_path, f"{modality}.csv")
          if not os.path.exists(file_path):
              print(f"Création de {file_path} avec des données moyennes et structure correcte")
              shape = data_shapes[modality]
              synthetic_data = pd.DataFrame(np.tile(mean_data[modality].values, (shape[0], 1)), columns=mean_data[modality].columns)
              synthetic_data = synthetic_data.reindex(columns=column_structure[modality], fill_value=0)  # Alignement des colonnes
              synthetic_data.to_csv(file_path, index=False)

In [ ]:
# Calculer les moyennes à partir des participants 1 à 16
existing_participants = list(range(1, 17))

In [ ]:
# Créer les fichiers manquants pour chaque participant de la liste
#column_structure, data_shapes = get_column_structure_and_shape(data_path, existing_participants, activities, modalities)
#for participant_id in participants_manque_files:
    #mean_data = calculate_mean_structure(data_path, existing_participants, activities, modalities)
    #create_missing_files_with_mean(data_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes)

print("Tous les dossiers et fichiers manquants ont été créés avec des valeurs moyennes et une structure correcte.")

Tous les dossiers et fichiers manquants ont été créés avec des valeurs moyennes et une structure correcte.


In [ ]:
"""newFileips = pd.read_csv('/content/drive/MyDrive/MovePort/18/forward/ips_1.csv')
newFileemg = pd.read_csv('/content/drive/MyDrive/MovePort/19/forward/emg_1.csv')
newFileimu = pd.read_csv('/content/drive/MyDrive/MovePort/21/forward/imu_1.csv')
print('shape of ips' ,newFileips.shape)
print('shape of emg' ,newFileemg.shape)
print('shape of imu' ,newFileimu.shape)"""

"newFileips = pd.read_csv('/content/drive/MyDrive/MovePort/18/forward/ips_1.csv')\nnewFileemg = pd.read_csv('/content/drive/MyDrive/MovePort/19/forward/emg_1.csv')\nnewFileimu = pd.read_csv('/content/drive/MyDrive/MovePort/21/forward/imu_1.csv')\nprint('shape of ips' ,newFileips.shape)\nprint('shape of emg' ,newFileemg.shape)\nprint('shape of imu' ,newFileimu.shape)"

In [ ]:
# justqu'au la c'est carré !

In [4]:
from sklearn.preprocessing import LabelEncoder
# Encoder les catégories
categories = ['back', 'forward', 'halfsquat', 'still']
category_encoder = LabelEncoder()
category_encoder.fit(categories)

LabelEncoder()

In [5]:
def decode_categories(df, column_name='categorie'):
  df[column_name] = category_encoder.inverse_transform(df[column_name])
  return df

In [6]:
def inverser_csv(file_path):
  df = pd.read_csv(file_path, header=0)
  df = df.transpose()
  df.columns = df.iloc[0]  # Première ligne devient l'en-tête
  df = df[1:].reset_index(drop=True)  # Supprimer la première ligne et réindexer
  return df

In [7]:
def process_data_in_batch(participant_path, modality):
  # Dictionnaire pour mapper les catégories aux valeurs numériques manuellement
  # category_mapping = {'back': 0, 'forward': 1, 'halfsquat': 2, 'still': 3}
  all_category_data = []

  for category in categories:
      # Chercher les fichiers spécifiques à la modalité
      files = glob.glob(os.path.join(participant_path, category, f"{modality}_1.csv"))
      print(f"  Fichiers {modality.upper()} trouvés pour la catégorie '{category}': {files}")
      category_data = []  # Liste pour stocker les données de cette catégorie
      for file in files:
          print(f"    Traitement du fichier {modality.upper()} : {file}")
          try:
              df = inverser_csv(file)
              if not df.index.is_unique:
                #firstCol = df.iloc[:, 0]
                df = df.loc[~df.duplicated(keep='first')].reset_index(drop=True)
              df['categorie'] = category_encoder.transform([category])[0]  # Ajouter la catégorie encodée
              category_data.append(df)  # Ajouter au lot de la catégorie
          except Exception as e:
              print(f"Erreur lors du traitement du fichier {file}: {e}")
      # Concaténer toutes les données de la catégorie pour ce participant
      if category_data:
          concatenated_category_data = pd.concat(category_data, ignore_index=True)
          all_category_data.append(concatenated_category_data)
  # Retourner toutes les données des catégories pour ce participant
  if all_category_data:
      return pd.concat(all_category_data, ignore_index=True)
  else:
      return None

In [8]:
def process_participants(data_path, participants, modality, batch_size=5):

  all_data = []  # Liste pour stocker toutes les données traitées
  # Diviser les participants en lots (batchs)
  participants_batches = [participants[i:i + batch_size] for i in range(0, len(participants), batch_size)]

  for batch_idx, batch_participants in enumerate(participants_batches, start=1):
      print(f"Traitement du lot {batch_idx}/{len(participants_batches)} : Participants {batch_participants}")
      batch_data = []  # Liste pour stocker les données d'un batch
      for participant_id in batch_participants:
          participant_path = os.path.join(data_path, str(participant_id))
          # Traiter les données pour ce participant
          print(f"Participant {participant_id}")
          participant_data = process_data_in_batch(participant_path, modality)
          # Ajouter l'ID du participant aux données
          if participant_data is not None:
              participant_data['id'] = participant_id  # Ajouter l'ID du participant
              batch_data.append(participant_data)
      # Concaténer les données du lot
      if batch_data:
          batch_df = pd.concat(batch_data, ignore_index=True)
          all_data.append(batch_df)
  # Concaténer toutes les données traitées en un seul DataFrame
  if all_data:
      concatenated_data = pd.concat(all_data, ignore_index=True)
      return concatenated_data
  else:
      print("Aucune donnée valide trouvée.")
      return None

In [9]:
batch_size = 5  # Taille du lot
participants = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#participants = participants

In [10]:
modalityImu = 'imu'  # Modalité, ici imu
modalityEmg = 'emg'  # Modalité, ici emg
modalityIps = 'ips'  # Modalité, ici ips
# Appel de la fonction pour traiter les données
processed_dataImu = process_participants(data_path, participants, modalityImu, batch_size)
processed_dataEmg = process_participants(data_path, participants, modalityEmg, batch_size)
processed_dataIps = process_participants(data_path, participants, modalityIps, batch_size)

Traitement du lot 1/2 : Participants [1, 2, 3, 4, 5]
Participant 1
  Fichiers IMU trouvés pour la catégorie 'back': ['/content/drive/MyDrive/MovePort/1/back/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/back/imu_1.csv
  Fichiers IMU trouvés pour la catégorie 'forward': ['/content/drive/MyDrive/MovePort/1/forward/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/forward/imu_1.csv
  Fichiers IMU trouvés pour la catégorie 'halfsquat': ['/content/drive/MyDrive/MovePort/1/halfsquat/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/halfsquat/imu_1.csv
  Fichiers IMU trouvés pour la catégorie 'still': ['/content/drive/MyDrive/MovePort/1/still/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/still/imu_1.csv
Participant 2
  Fichiers IMU trouvés pour la catégorie 'back': ['/content/drive/MyDrive/MovePort/2/back/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/Mov

In [11]:
print("imu procces shape : ", processed_dataImu.shape)
print("emg procces shape : ", processed_dataEmg.shape)
print("ips procces shape : ", processed_dataIps.shape)

imu procces shape :  (244072, 56)
emg procces shape :  (4881402, 18)
ips procces shape :  (152975, 684)


In [12]:
"""print("imu procces nulls : ", processed_dataImu.isnull().sum())
print("emg procces nulls : ", processed_dataEmg.isnull().sum())
print("ips procces nulls : ", processed_dataIps.isnull().sum())"""

'print("imu procces nulls : ", processed_dataImu.isnull().sum())\nprint("emg procces nulls : ", processed_dataEmg.isnull().sum())\nprint("ips procces nulls : ", processed_dataIps.isnull().sum())'

In [13]:
def convert_object_to_numeric(data):
  for col in data.columns:
      if data[col].dtype == 'object':
          data[col] = pd.to_numeric(data[col], errors='coerce')
  return data

In [14]:
"""print("imu procces types : ", processed_dataImu.dtypes)
print("emg procces types : ", processed_dataEmg.dtypes)
print("ips procces types : ", processed_dataIps.dtypes)"""

'print("imu procces types : ", processed_dataImu.dtypes)\nprint("emg procces types : ", processed_dataEmg.dtypes)\nprint("ips procces types : ", processed_dataIps.dtypes)'

In [15]:
processed_dataImu = convert_object_to_numeric(processed_dataImu)
processed_dataEmg = convert_object_to_numeric(processed_dataEmg)
processed_dataIps = convert_object_to_numeric(processed_dataIps)

In [16]:
print("imu procces types : ", processed_dataImu.dtypes)
print("emg procces types : ", processed_dataEmg.dtypes)
print("ips procces types : ", processed_dataIps.dtypes)

imu procces types :  Unnamed: 0
Head_Acc_X     float64
Head_Acc_Y     float64
Head_Acc_Z     float64
Head_Gyr_X     float64
Head_Gyr_Y     float64
Head_Gyr_Z     float64
Head_Roll      float64
Head_Pitch     float64
Head_Yaw       float64
Waist_Acc_X    float64
Waist_Acc_Y    float64
Waist_Acc_Z    float64
Waist_Gyr_X    float64
Waist_Gyr_Y    float64
Waist_Gyr_Z    float64
Waist_Roll     float64
Waist_Pitch    float64
Waist_Yaw      float64
L_H_Acc_X      float64
L_H_Acc_Y      float64
L_H_Acc_Z      float64
L_H_Gyr_X      float64
L_H_Gyr_Y      float64
L_H_Gyr_Z      float64
L_H_Roll       float64
L_H_Pitch      float64
L_H_Yaw        float64
R_H_Acc_X      float64
R_H_Acc_Y      float64
R_H_Acc_Z      float64
R_H_Gyr_X      float64
R_H_Gyr_Y      float64
R_H_Gyr_Z      float64
R_H_Roll       float64
R_H_Pitch      float64
R_H_Yaw        float64
L_F_Acc_X      float64
L_F_Acc_Y      float64
L_F_Acc_Z      float64
L_F_Gyr_X      float64
L_F_Gyr_Y      float64
L_F_Gyr_Z      float64
L_

In [19]:
# une etape important pour la memoire !!!
# Les colonnes de type float64 et int64 occupent beaucoup de mémoire. convertir en types plus compacts (float32 ou int32)
def reduce_memory_usage(df):
  for col in df.columns:
      col_type = df[col].dtype
      if col_type == 'float64':
          df[col] = df[col].astype('float32')
      elif col_type == 'int64':
          df[col] = df[col].astype('int32')
  return df

In [20]:
processed_dataImu = reduce_memory_usage(processed_dataImu)
processed_dataEmg = reduce_memory_usage(processed_dataEmg)
processed_dataIps = reduce_memory_usage(processed_dataIps)

In [21]:
print("imu procces types : ", processed_dataImu.dtypes)
print("emg procces types : ", processed_dataEmg.dtypes)
print("ips procces types : ", processed_dataIps.dtypes)

imu procces types :  Unnamed: 0
Head_Acc_X     float32
Head_Acc_Y     float32
Head_Acc_Z     float32
Head_Gyr_X     float32
Head_Gyr_Y     float32
Head_Gyr_Z     float32
Head_Roll      float32
Head_Pitch     float32
Head_Yaw       float32
Waist_Acc_X    float32
Waist_Acc_Y    float32
Waist_Acc_Z    float32
Waist_Gyr_X    float32
Waist_Gyr_Y    float32
Waist_Gyr_Z    float32
Waist_Roll     float32
Waist_Pitch    float32
Waist_Yaw      float32
L_H_Acc_X      float32
L_H_Acc_Y      float32
L_H_Acc_Z      float32
L_H_Gyr_X      float32
L_H_Gyr_Y      float32
L_H_Gyr_Z      float32
L_H_Roll       float32
L_H_Pitch      float32
L_H_Yaw        float32
R_H_Acc_X      float32
R_H_Acc_Y      float32
R_H_Acc_Z      float32
R_H_Gyr_X      float32
R_H_Gyr_Y      float32
R_H_Gyr_Z      float32
R_H_Roll       float32
R_H_Pitch      float32
R_H_Yaw        float32
L_F_Acc_X      float32
L_F_Acc_Y      float32
L_F_Acc_Z      float32
L_F_Gyr_X      float32
L_F_Gyr_Y      float32
L_F_Gyr_Z      float32
L_

In [22]:
def clean_missing_values(processed_data, threshold=0.5):

  zero_threshold = (processed_data == 0).mean(axis=1)  # Calcul du pourcentage de zéros par ligne
  rows_to_delete = zero_threshold[zero_threshold >= threshold]
  #print(f"Lignes supprimées (trop de zéros):\n{rows_to_delete}")
  processed_data = processed_data.loc[zero_threshold < threshold]
  print(f"Nombre de lignes avant suppression: {len(zero_threshold)}")
  print(f"Nombre de lignes après suppression: {len(processed_data)}")
  numeric_cols = processed_data.select_dtypes(include=[np.number])
  non_numeric_cols = processed_data.select_dtypes(exclude=[np.number])
  missing_percentage = numeric_cols.isna().mean()
  numeric_cols_cleaned = numeric_cols.loc[:, missing_percentage < threshold]
  numeric_cols_cleaned = numeric_cols_cleaned.apply(lambda col: col.fillna(col.median()), axis=0)
  processed_data_cleaned = pd.concat([non_numeric_cols, numeric_cols_cleaned], axis=1)

  return processed_data_cleaned

In [23]:
#cleaned_data = clean_missing_values(processed_data, porcentage 0.5)
cleaned_dataImu = clean_missing_values(processed_dataImu, threshold=0.5)
cleaned_dataEmg = clean_missing_values(processed_dataEmg, threshold=0.5)
cleaned_dataIps = clean_missing_values(processed_dataIps, threshold=0.5)

Nombre de lignes avant suppression: 244072
Nombre de lignes après suppression: 244020
Nombre de lignes avant suppression: 4881402
Nombre de lignes après suppression: 4881402
Nombre de lignes avant suppression: 152975
Nombre de lignes après suppression: 70983


In [24]:
#cleaned_data.shape
print("cleaned_cleaned Imu : ", cleaned_dataImu.shape)
print("cleaned_cleaned Emg : ", cleaned_dataEmg.shape)
print("cleaned_cleaned ips : ", cleaned_dataIps.shape)

cleaned_cleaned Imu :  (244020, 56)
cleaned_cleaned Emg :  (4881402, 18)
cleaned_cleaned ips :  (70983, 684)


In [42]:
cleaned_dataEmg.head(3)

Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,categorie,id
time,,,,,,,,,,,,,,,,,,
2024-11-24 00:00:00.000000,-0.704956,1.309204,-2.316284,-20.242310,-6.747437,2.920532,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1
2024-11-24 00:00:00.000500,2.014160,1.309204,-2.316284,-16.415405,-1.208496,2.618408,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1
2024-11-24 00:00:00.001000,-0.704956,1.309204,-2.316284,-16.314697,-1.107788,1.107788,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1


In [82]:
print("Catégories dans IPS:", cleaned_dataIps['categorie'].unique())
print("Catégories dans IMU:", cleaned_dataImu['categorie'].unique())
print("Catégories dans EMG:", cleaned_dataEmg['categorie'].unique())

Catégories dans IPS: [0 1 2 3]
Catégories dans IMU: [0 1 2 3]
Catégories dans EMG: [0 1 2 3]


In [ ]:
# juste for test
"""def estimate_frequency(sample_count, duration_seconds):

    freq_hz = sample_count / duration_seconds
    return freq_hz
emg_sample_count = cleaned_dataEmg.shape[0]
imu_sample_count = cleaned_dataImu.shape[0]
ips_sample_count = cleaned_dataIps.shape[0]

emg_freq = estimate_frequency(emg_sample_count, 2000)
imu_freq = estimate_frequency(imu_sample_count, 100)
ips_freq = estimate_frequency(ips_sample_count, 35)

print(f"EMG Frequency: {emg_freq:.2f} Hz")
print(f"IMU Frequency: {imu_freq:.2f} Hz")
print(f"IPS Frequency: {ips_freq:.2f} Hz")"""


'def estimate_frequency(sample_count, duration_seconds):\n\n    freq_hz = sample_count / duration_seconds\n    return freq_hz\nemg_sample_count = cleaned_dataEmg.shape[0]\nimu_sample_count = cleaned_dataImu.shape[0]\nips_sample_count = cleaned_dataIps.shape[0]\n\nemg_freq = estimate_frequency(emg_sample_count, 2000)\nimu_freq = estimate_frequency(imu_sample_count, 100)\nips_freq = estimate_frequency(ips_sample_count, 35)\n\nprint(f"EMG Frequency: {emg_freq:.2f} Hz")\nprint(f"IMU Frequency: {imu_freq:.2f} Hz")\nprint(f"IPS Frequency: {ips_freq:.2f} Hz")'

In [83]:
"""def resample_data(df, original_freq, target_freq, debug=False):

  if original_freq <= 0 or target_freq <= 0:
      raise ValueError("Les fréquences d'échantillonnage doivent être positives et non nulles.")
  if original_freq == target_freq:
      return df
  if debug:
      print(f"Rééchantillonnage de {original_freq} Hz à {target_freq} Hz.")
  if 'time' not in df.columns:
      df['time'] = pd.date_range(
          start='00:00:00',
          periods=df.shape[0],
          freq=f'{int(1e6 / original_freq)}us'
      )
  df.set_index('time', inplace=True)
  resampled_df = df.resample(f'{int(1e6 / target_freq)}us').interpolate(method='linear')
  resampled_df.reset_index(inplace=True)
  resampled_df.drop(columns=['time'], inplace=True, errors='ignore')
  if debug:
      print(f"Nombre de points après interpolation : {resampled_df.shape[0]}")

  return resampled_df"""

'def resample_data(df, original_freq, target_freq, debug=False):\n\n  if original_freq <= 0 or target_freq <= 0:\n      raise ValueError("Les fréquences d\'échantillonnage doivent être positives et non nulles.")\n  if original_freq == target_freq:\n      return df\n  if debug:\n      print(f"Rééchantillonnage de {original_freq} Hz à {target_freq} Hz.")\n  if \'time\' not in df.columns:\n      df[\'time\'] = pd.date_range(\n          start=\'00:00:00\',\n          periods=df.shape[0],\n          freq=f\'{int(1e6 / original_freq)}us\'\n      )\n  df.set_index(\'time\', inplace=True)\n  resampled_df = df.resample(f\'{int(1e6 / target_freq)}us\').interpolate(method=\'linear\')\n  resampled_df.reset_index(inplace=True)\n  resampled_df.drop(columns=[\'time\'], inplace=True, errors=\'ignore\')\n  if debug:\n      print(f"Nombre de points après interpolation : {resampled_df.shape[0]}")\n\n  return resampled_df'

In [87]:
def resample_data(df, original_freq, target_freq, ignore_columns=['categorie'], debug=False):
  if original_freq <= 0 or target_freq <= 0:
      raise ValueError("Les fréquences d'échantillonnage doivent être positives et non nulles.")
  if original_freq == target_freq:
      return df
  if debug:
      print(f"Rééchantillonnage de {original_freq} Hz à {target_freq} Hz.")

  ignore_columns = [col for col in ignore_columns if col in df.columns]
  columns_to_resample = df.drop(columns=ignore_columns)

  df['time'] = pd.date_range(
      start='00:00:00',
      periods=columns_to_resample.shape[0],
      freq=f'{int(1e6 / original_freq)}us'
  )
  df.set_index('time', inplace=True)

  resampled_df = columns_to_resample.resample(f'{int(1e6 / target_freq)}us').interpolate(method='linear')

  for col in ignore_columns:
      resampled_df[col] = df[col].resample(f'{int(1e6 / target_freq)}us').ffill()

  resampled_df.reset_index(inplace=True)
  resampled_df.drop(columns=['time'], inplace=True, errors='ignore')

  if debug:
      print(f"Nombre de points après interpolation : {resampled_df.shape[0]}")

  return resampled_df


In [52]:
"""def plot_data_comparison(original_df, resampled_df, modality_name):

  plt.figure(figsize=(14, 6))
  # Comparaison de la première colonne
  original_column = original_df.columns[0]
  resampled_column = resampled_df.columns[0]
  # Plot des données originales
  plt.subplot(1, 2, 1)
  plt.plot(original_df.index, original_df[original_column], label='Original')
  plt.title(f'{modality_name} - Original Data')
  plt.xlabel('Time Index')
  plt.ylabel('Value')
  plt.legend()
  # Plot des données rééchantillonnées
  plt.subplot(1, 2, 2)
  plt.plot(resampled_df.index, resampled_df[resampled_column], label='Resampled', color='orange')
  plt.title(f'{modality_name} - Resampled Data')
  plt.xlabel('Time Index')
  plt.ylabel('Value')
  plt.legend()

  plt.tight_layout()
  plt.show()"""

"def plot_data_comparison(original_df, resampled_df, modality_name):\n\n  plt.figure(figsize=(14, 6))\n  # Comparaison de la première colonne\n  original_column = original_df.columns[0]\n  resampled_column = resampled_df.columns[0]\n  # Plot des données originales\n  plt.subplot(1, 2, 1)\n  plt.plot(original_df.index, original_df[original_column], label='Original')\n  plt.title(f'{modality_name} - Original Data')\n  plt.xlabel('Time Index')\n  plt.ylabel('Value')\n  plt.legend()\n  # Plot des données rééchantillonnées\n  plt.subplot(1, 2, 2)\n  plt.plot(resampled_df.index, resampled_df[resampled_column], label='Resampled', color='orange')\n  plt.title(f'{modality_name} - Resampled Data')\n  plt.xlabel('Time Index')\n  plt.ylabel('Value')\n  plt.legend()\n\n  plt.tight_layout()\n  plt.show()"

In [88]:
emg_resampled = resample_data(cleaned_dataEmg, original_freq=2000, target_freq=60)
imu_resampled = resample_data(cleaned_dataImu, original_freq=100, target_freq=60)
ips_resampled = resample_data(cleaned_dataIps, original_freq=60, target_freq=60)

In [89]:
print("rééchantillonné Imu : ", imu_resampled.shape)
print("rééchantillonné Emg : ", emg_resampled.shape)
print("rééchantillonné ips : ", ips_resampled.shape)

rééchantillonné Imu :  (146418, 56)
rééchantillonné Emg :  (146448, 18)
rééchantillonné ips :  (70983, 684)


In [90]:
print("Catégories dans IPS:", ips_resampled['categorie'].unique())
print("Catégories dans IMU:", imu_resampled['categorie'].unique())
print("Catégories dans EMG:", emg_resampled['categorie'].unique())

Catégories dans IPS: [0 1 2 3]
Catégories dans IMU: [0 1 2 3]
Catégories dans EMG: [0 1 2 3]


In [ ]:
"""plot_data_comparison(cleaned_dataEmg, emg_resampled, 'EMG')
plot_data_comparison(cleaned_dataImu, imu_resampled, 'IMU')
plot_data_comparison(cleaned_dataIps, ips_resampled, 'IPS')"""

"plot_data_comparison(cleaned_dataEmg, emg_resampled, 'EMG')\nplot_data_comparison(cleaned_dataImu, imu_resampled, 'IMU')\nplot_data_comparison(cleaned_dataIps, ips_resampled, 'IPS')"

In [91]:
imu_resampled.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,id,categorie
0,8.902975,0.412529,4.052022,-0.015015,0.010260,-0.009929,-12.639535,-10.234849,30.077293,9.578385,...,-1.903201,9.019931,-0.027493,0.009596,-0.008538,10.781965,35.531990,-17.776943,1.0,0
1,8.902772,0.412443,4.052364,-0.015010,0.010255,-0.009926,-12.641062,-10.232953,30.077219,9.578046,...,-1.903192,9.019924,-0.027488,0.009595,-0.008536,10.782005,35.531818,-17.774742,1.0,0
2,8.902568,0.412356,4.052706,-0.015005,0.010250,-0.009923,-12.642589,-10.231056,30.077147,9.577707,...,-1.903184,9.019917,-0.027483,0.009593,-0.008533,10.782044,35.531651,-17.772543,1.0,0


In [92]:
def normalize_data(data, ignore_columns=['id', 'categorie']):
  # Vérification des colonnes à ignorer
  ignore_columns = [col for col in ignore_columns if col in data.columns]
  if len(ignore_columns) < len(set(ignore_columns)):
    print("alert : certaines colonnes dans 'ignore_columns' n'existent pas dans le DataFrame.")
  # Séparer les colonnes à ignorer (id, categorie, etc.)
  to_ignore = data[ignore_columns]
  to_normalize = data.drop(columns=ignore_columns)
  numeric_cols = to_normalize.select_dtypes(include=[float, int]).drop(columns=['categorie'], errors='ignore')
  if 'categorie' in numeric_cols.columns:
    numeric_cols['categorie'] = numeric_cols['categorie'].astype('object')
  # Normalisation des colonnes numériques
  scaler = StandardScaler()
  normalized_numeric = pd.DataFrame(
      scaler.fit_transform(numeric_cols),
      columns=numeric_cols.columns,
      index=numeric_cols.index
  )
  # Réintégration des colonnes non numériques ignorées
  non_numeric_cols = to_normalize.select_dtypes(exclude=[float, int])
  if not non_numeric_cols.empty:
    print("Notif : Les colonnes non numériques ont été exclues de la normalisation.")
  # Reconstruction du DataFrame final
  normalized_data = pd.concat([normalized_numeric.reset_index(drop=True), non_numeric_cols.reset_index(drop=True), to_ignore.reset_index(drop=True)], axis=1)
  # Restaurer l'ordre original des colonnes
  normalized_data = normalized_data[data.columns]

  return normalized_data

In [93]:
normalized_dataImu = normalize_data(imu_resampled)
normalized_dataEmg = normalize_data(emg_resampled)
normalized_dataIps = normalize_data(ips_resampled)

In [94]:
normalized_dataImu.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,id,categorie
0,0.148848,0.466966,0.750972,-0.343638,-0.140327,-0.390197,0.181553,0.218360,0.714770,0.579785,...,-1.433016,1.689364,-1.289585,0.654196,-1.774494,1.743718,2.010086,0.215327,1.0,0
1,0.148692,0.466865,0.751080,-0.343531,-0.140362,-0.390162,0.181509,0.218431,0.714768,0.579703,...,-1.433008,1.689344,-1.289287,0.654093,-1.774156,1.743725,2.010075,0.215369,1.0,0
2,0.148535,0.466764,0.751187,-0.343423,-0.140396,-0.390128,0.181465,0.218502,0.714767,0.579621,...,-1.433001,1.689321,-1.288990,0.653991,-1.773818,1.743732,2.010064,0.215410,1.0,0


In [95]:
normalized_dataEmg.head(3)

Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,id,categorie
0,-0.007780,0.16006,0.087106,-0.339345,-0.121900,0.113590,0.089586,-0.084776,0.104333,-1.369259,2.554450,-0.01682,0.074650,0.598489,-0.345091,0.277640,1.0,0
1,-0.008057,0.16119,0.086823,-0.336059,-0.122467,0.111774,0.090539,-0.090888,0.103300,-1.361942,2.545334,-0.01767,0.074963,0.597330,-0.344909,0.277656,1.0,0
2,-0.008335,0.16232,0.086539,-0.332774,-0.123033,0.109958,0.091492,-0.097001,0.102268,-1.354626,2.536217,-0.01852,0.075276,0.596171,-0.344727,0.277672,1.0,0


In [96]:
normalized_dataIps.head(3)

NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,674.0,675.0,676.0,677.0,678.0,679.0,680.0,681.0,categorie,id
0,0.0,0.0,0.0,0.0,0.0,-0.52313,-0.471738,0.367731,-0.623475,0.0,...,0.0,0.0,0.0,-0.585282,-0.49221,-0.417457,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,0.0,-0.52313,-0.471738,0.245660,-0.623475,0.0,...,0.0,0.0,0.0,-0.585282,-0.49221,-0.417457,0.0,0.0,0,1
2,0.0,0.0,0.0,0.0,0.0,-0.52313,-0.471738,0.367731,-0.623475,0.0,...,0.0,0.0,0.0,-0.585282,-0.49221,-0.417457,0.0,0.0,0,1


sauvgarder les modalities !!

In [5]:
processed_data_path = '/content/drive/MyDrive/MovePort/processedData'
if not os.path.exists(processed_data_path): os.makedirs(processed_data_path)

In [98]:
emg_normalized = pd.DataFrame(normalized_dataEmg)
imu_normalized = pd.DataFrame(normalized_dataImu)
ips_normalized = pd.DataFrame(normalized_dataIps)

In [99]:
emg_normalized.to_csv(os.path.join(processed_data_path, 'emg_normalized.csv'), index=False)
imu_normalized.to_csv(os.path.join(processed_data_path, 'imu_normalized.csv'), index=False)
ips_normalized.to_csv(os.path.join(processed_data_path, 'ips_normalized.csv'), index=False)

In [6]:
emg_normalized = pd.read_csv(f"{processed_data_path}/emg_normalized.csv")
imu_normalized = pd.read_csv(f"{processed_data_path}/imu_normalized.csv")
ips_normalized = pd.read_csv(f"{processed_data_path}/ips_normalized.csv")

In [59]:
#imu_normalized.value_counts()

In [7]:
emg_normalized.head(3)

,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,id,categorie
0,-0.007780,0.16006,0.087106,-0.339345,-0.121900,0.113590,0.089586,-0.084776,0.104333,-1.369259,2.554450,-0.01682,0.074650,0.598489,-0.345091,0.277640,1.0,0
1,-0.008057,0.16119,0.086823,-0.336059,-0.122467,0.111774,0.090539,-0.090888,0.103300,-1.361942,2.545334,-0.01767,0.074963,0.597330,-0.344909,0.277656,1.0,0
2,-0.008335,0.16232,0.086539,-0.332774,-0.123033,0.109958,0.091492,-0.097001,0.102268,-1.354626,2.536217,-0.01852,0.075276,0.596171,-0.344727,0.277672,1.0,0


In [8]:
print("Valeurs uniques dans EMG avant conversion :", emg_normalized['categorie'].unique())
print("Valeurs uniques dans EMG après conversion :", emg_normalized['categorie'].astype(int).unique())

Valeurs uniques dans EMG avant conversion : [0 1 2 3]
Valeurs uniques dans EMG après conversion : [0 1 2 3]


In [9]:
print("Distribution des catégories EMG avant conversion :")
print(emg_normalized['categorie'].value_counts())
print("Distribution des catégories EMG après conversion :")
print(emg_normalized['categorie'].astype(int).value_counts())


Distribution des catégories EMG avant conversion :
categorie
0    38376
2    37580
3    35309
1    35183
Name: count, dtype: int64
Distribution des catégories EMG après conversion :
categorie
0    38376
2    37580
3    35309
1    35183
Name: count, dtype: int64


In [10]:
"""print("Lignes avec categorie = 2 dans EMG :")
print(emg_normalized[emg_normalized['categorie'] == 2])
print("Lignes avec categorie = 3 dans EMG :")
print(emg_normalized[emg_normalized['categorie'] == 3])"""

'print("Lignes avec categorie = 2 dans EMG :")\nprint(emg_normalized[emg_normalized[\'categorie\'] == 2])\nprint("Lignes avec categorie = 3 dans EMG :")\nprint(emg_normalized[emg_normalized[\'categorie\'] == 3])'

In [11]:
print("Catégories dans IPS:", ips_normalized['categorie'].unique())
print("Catégories dans IMU:", imu_normalized['categorie'].unique())
print("Catégories dans EMG:", emg_normalized['categorie'].unique())

Catégories dans IPS: [0 1 2 3]
Catégories dans IMU: [0 1 2 3]
Catégories dans EMG: [0 1 2 3]


In [12]:
print("Type de 'categorie' dans IPS:", ips_normalized['categorie'].dtype)
print("Type de 'categorie' dans IMU:", imu_normalized['categorie'].dtype)
print("Type de 'categorie' dans EMG:", emg_normalized['categorie'].dtype)

Type de 'categorie' dans IPS: int64
Type de 'categorie' dans IMU: int64
Type de 'categorie' dans EMG: int64


In [107]:
"""# Corriger le type de 'categorie' après le rééchantillonnage
ips_normalized['categorie'] = ips_normalized['categorie'].astype(int)
imu_normalized['categorie'] = imu_normalized['categorie'].astype(int)
emg_normalized['categorie'] = emg_normalized['categorie'].astype(int)"""

"# Corriger le type de 'categorie' après le rééchantillonnage\nips_normalized['categorie'] = ips_normalized['categorie'].astype(int)\nimu_normalized['categorie'] = imu_normalized['categorie'].astype(int)\nemg_normalized['categorie'] = emg_normalized['categorie'].astype(int)"

In [13]:
# Compter les catégories dans chaque modalité avant normalisation ou autre traitement
print("Avant traitement :")
print("IPS Categories Count:", ips_normalized['categorie'].value_counts())
print("IMU Categories Count:", imu_normalized['categorie'].value_counts())
print("EMG Categories Count:", emg_normalized['categorie'].value_counts())

Avant traitement :
IPS Categories Count: categorie
3    30704
1    23878
2    13499
0     2902
Name: count, dtype: int64
IMU Categories Count: categorie
0    38372
2    37575
3    35297
1    35174
Name: count, dtype: int64
EMG Categories Count: categorie
0    38376
2    37580
3    35309
1    35183
Name: count, dtype: int64


In [14]:
def calculate_category_distribution(modality_data, target_column='categorie'):
  return modality_data[target_column].value_counts()

ips_distribution = calculate_category_distribution(ips_normalized, target_column='categorie')
imu_distribution = calculate_category_distribution(imu_normalized, target_column='categorie')
emg_distribution = calculate_category_distribution(emg_normalized, target_column='categorie')

distribution_table = pd.DataFrame({
    'ips': ips_distribution,
    'imu': imu_distribution,
    'emg': emg_distribution
}).fillna(0).astype(int)

print(distribution_table)


             ips    imu    emg
categorie                     
0           2902  38372  38376
1          23878  35174  35183
2          13499  37575  37580
3          30704  35297  35309


In [15]:
numbreMean = pd.DataFrame(distribution_table, index=[0, 1, 2, 3])
numbreMean

,ips,imu,emg
0,2902,38372,38376
1,23878,35174,35183
2,13499,37575,37580
3,30704,35297,35309


In [17]:
moyenne_min = numbreMean.mean().min()
moyenne_min

17745.75

In [26]:
valeur_arrondie = int(round(moyenne_min, -3))  # Arrondir à la centaine la plus proche
print("Valeur arrondie la plus proche:", valeur_arrondie)

Valeur arrondie la plus proche: 18000


In [27]:
from sklearn.utils import resample
def align_modalities_to_target(data, target_column, target_size, random_state=42):

  aligned_data = []
  # Parcourir chaque catégorie
  for category in data[target_column].unique():
      category_data = data[data[target_column] == category]
      current_size = len(category_data)

      if current_size > target_size:
          # Réduire si la catégorie a plus de données que la cible
          reduced_data = category_data.sample(n=target_size, random_state=random_state)
          aligned_data.append(reduced_data)
      elif current_size < target_size:
          # Augmenter si la catégorie a moins de données que la cible
          increased_data = resample(
              category_data,
              replace=True,  # Over-sampling
              n_samples=target_size,
              random_state=random_state
          )
          aligned_data.append(increased_data)
      else:
          # Si déjà aligné, conserver tel quel
          aligned_data.append(category_data)

  return pd.concat(aligned_data).reset_index(drop=True)

In [55]:
print("normilized Imu : ", imu_normalized.shape)
print("normilized Emg : ", emg_normalized.shape)
print("normilized ips : ", ips_normalized.shape)

normilized Imu :  (146418, 56)
normilized Emg :  (146448, 18)
normilized ips :  (70983, 684)


In [29]:
ips_aligned = align_modalities_to_target(ips_normalized, target_column='categorie', target_size=valeur_arrondie)
imu_aligned = align_modalities_to_target(imu_normalized, target_column='categorie', target_size=valeur_arrondie)
emg_aligned = align_modalities_to_target(emg_normalized, target_column='categorie', target_size=valeur_arrondie)

In [54]:
print("aligned Imu : ", imu_aligned.shape)
print("aligned Emg : ", emg_aligned.shape)
print("aligned ips : ", ips_aligned.shape)

aligned Imu :  (72000, 56)
aligned Emg :  (72000, 18)
aligned ips :  (72000, 684)


In [32]:
ips_distribution = calculate_category_distribution(ips_aligned, target_column='categorie')
imu_distribution = calculate_category_distribution(imu_aligned, target_column='categorie')
emg_distribution = calculate_category_distribution(emg_aligned, target_column='categorie')

distribution_table = pd.DataFrame({
    'ips': ips_distribution,
    'imu': imu_distribution,
    'emg': emg_distribution
}).fillna(0).astype(int)

print(distribution_table)

             ips    imu    emg
categorie                     
0          18000  18000  18000
1          18000  18000  18000
2          18000  18000  18000
3          18000  18000  18000


In [33]:
# Concaténation des modalités alignées
all_aligned_data = pd.concat([ips_aligned, imu_aligned, emg_aligned], axis=0)
# Vérification des distributions
print("Final Category Distribution Across Modalities:")
print(all_aligned_data['categorie'].value_counts())

Final Category Distribution Across Modalities:
categorie
0    54000
1    54000
2    54000
3    54000
Name: count, dtype: int64


In [36]:
emg_aligned.to_csv(os.path.join(processed_data_path, 'emg_aligned.csv'), index=False)
imu_aligned.to_csv(os.path.join(processed_data_path, 'imu_aligned.csv'), index=False)
ips_aligned.to_csv(os.path.join(processed_data_path, 'ips_aligned.csv'), index=False)

In [34]:
all_aligned_data.to_csv(os.path.join(processed_data_path, 'all_aligned_data.csv'), index=False)

In [38]:
emg_aligned = pd.read_csv(f"{processed_data_path}/emg_aligned.csv")
imu_aligned = pd.read_csv(f"{processed_data_path}/imu_aligned.csv")
ips_aligned = pd.read_csv(f"{processed_data_path}/ips_aligned.csv")

In [39]:
all_aligned_data = pd.read_csv(f"{processed_data_path}/all_aligned_data.csv")

In [50]:
# total colonnes doit être 752 + cat et id = 754
all_aligned_data.shape

(216000, 754)

In [40]:
def segment_data(data, window_size=100, step_size=50, target_column='categorie', ignore_columns=['id']):
  # Validation des paramètres
  if window_size <= 0 or step_size <= 0:
      raise ValueError("Les paramètres 'window_size' et 'step_size' doivent être positifs.")
  if target_column not in data.columns:
      raise KeyError(f"La colonne cible '{target_column}' n'existe pas dans les données.")
  if any(col not in data.columns for col in ignore_columns):
      raise KeyError(f"Certaines colonnes dans 'ignore_columns' n'existent pas dans les données.")
  if len(data) < window_size:
      raise ValueError("La taille des données est inférieure à 'window_size'.")

  # Séparer les colonnes numériques, la cible et les colonnes ignorées
  target = data[target_column]
  to_ignore = data[ignore_columns]
  data_numeric = data.drop(columns=[target_column] + ignore_columns)
  # Initialisation des segments et métadonnées
  segments = []
  targets = []
  ids = []
  # Création des fenêtres
  for start in range(0, len(data_numeric) - window_size + 1, step_size):
      # Fenêtre de données numériques
      window = data_numeric.iloc[start:start + window_size].values
      segments.append(window)

      # Ajouter la valeur cible et l'identifiant associés à la fenêtre
      targets.append(target.iloc[start + window_size - 1])
      ids.append(to_ignore['id'].iloc[start + window_size - 1])

  # Conversion des segments en tableau numpy
  segments_array = np.array(segments)

  # Ajout des métadonnées
  metadata_array = np.array(list(zip(targets, ids)))
  metadata_array = metadata_array[:, np.newaxis, :]  # Étendre pour l'axe de la fenêtre
  metadata_array = np.tile(metadata_array, (1, window_size, 1))  # Répliquer sur la longueur de la fenêtre

  # Combinaison des données et des métadonnées
  combined_segments_with_metadata = np.concatenate((segments_array, metadata_array), axis=2)

  return combined_segments_with_metadata

In [41]:
print("rééchantillonné Imu : ", imu_aligned.shape)
print("rééchantillonné Emg : ", emg_aligned.shape)
print("rééchantillonné ips : ", ips_aligned.shape)

rééchantillonné Imu :  (72000, 56)
rééchantillonné Emg :  (72000, 18)
rééchantillonné ips :  (72000, 684)


In [42]:
# la segmentation pour chaque modalities
segmented_dataImu = segment_data(imu_aligned, window_size=100, step_size=50)
segmented_dataEmg = segment_data(emg_aligned, window_size=100, step_size=50)
segmented_dataIps = segment_data(ips_aligned, window_size=100, step_size=50)

In [44]:
print("segmented data Imu : ", segmented_dataImu.shape)
print("segmented data Emg : ", segmented_dataEmg.shape)
print("segmented data ips : ", segmented_dataIps.shape)

rééchantillonné Imu :  (1439, 100, 56)
rééchantillonné Emg :  (1439, 100, 18)
rééchantillonné ips :  (1439, 100, 684)


In [47]:
segmented_dataEmg = pd.DataFrame(segmented_dataEmg[0])
segmented_dataEmg.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.211447,-0.360838,-0.250198,-1.685796,2.850908,3.940966,0.279367,-0.002210,0.126843,-1.100252,0.054470,-0.225750,0.042635,0.327928,-0.173993,0.263670,0.0,2.0
1,-0.032130,0.423728,-0.002732,0.821166,0.638891,-0.717586,0.534603,-0.014669,0.115138,-0.400031,0.025755,0.381319,-0.253444,0.142602,-0.455233,0.289226,0.0,2.0
2,-0.558133,-0.928317,0.425590,0.038364,1.519914,-1.073757,0.095849,0.370001,0.107246,0.203071,0.396287,-0.792443,0.066346,-0.351621,-0.812976,0.214499,0.0,2.0


In [51]:
segmented_all_aligned_data = segment_data(all_aligned_data, window_size=100, step_size=50)

In [52]:
print("not segmented data all : ", all_aligned_data.shape)
print("segmented data all : ", segmented_all_aligned_data.shape)

not segmented data all :  (216000, 754)
segmented data all :  (4319, 100, 754)


In [ ]:
from sklearn.model_selection import train_test_split

# Architecture unifiée : concaténation des modalités
X_unified = segmented_all_aligned_data.drop(columns=['categorie', 'id']).values
y_unified = segmented_all_aligned_data['categorie'].values

# Architecture multi-modale : données séparées
X_ips = segmented_dataIps.drop(columns=['categorie', 'id']).values
X_imu = segmented_dataImu.drop(columns=['categorie', 'id']).values
X_emg = segmented_dataEmg.drop(columns=['categorie', 'id']).values
y_multimodal = segmented_dataEmg['categorie'].values

# Séparer les données en train/test pour les deux architectures
X_unified_train, X_unified_test, y_unified_train, y_unified_test = train_test_split(X_unified, y_unified, test_size=0.2, random_state=42)

X_ips_train, X_ips_test, y_ips_train, y_ips_test = train_test_split(X_ips, y_multimodal, test_size=0.2, random_state=42)
X_imu_train, X_imu_test, y_imu_train, y_imu_test = train_test_split(X_imu, y_multimodal, test_size=0.2, random_state=42)
X_emg_train, X_emg_test, y_emg_train, y_emg_test = train_test_split(X_emg, y_multimodal, test_size=0.2, random_state=42)


In [ ]:
# Reshape pour CNN/LSTM
X_unified_train = X_unified_train.reshape(X_unified_train.shape[0], 1, X_unified_train.shape[1])
X_unified_test = X_unified_test.reshape(X_unified_test.shape[0], 1, X_unified_test.shape[1])


In [53]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, LSTM, Flatten, Dropout

# Modèle CNN + LSTM pour architecture unifiée
model_unified = Sequential()

# Couche CNN
model_unified.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_unified_train.shape[1], X_unified_train.shape[2])))
model_unified.add(MaxPooling1D(pool_size=2))
model_unified.add(Dropout(0.2))

# Couche LSTM
model_unified.add(LSTM(50, activation='relu', return_sequences=True))
model_unified.add(LSTM(50, activation='relu'))

# Couche Dense finale
model_unified.add(Dense(100, activation='relu'))
model_unified.add(Dense(len(set(y_unified)), activation='softmax'))  # Softmax pour classification

# Compilation
model_unified.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model_unified.summary()


In [114]:
"""def align_data_to_median(modalities):

  # Obtenir le nombre de fenêtres pour chaque modalité
  window_counts = [modality.shape[0] for modality in modalities]
  # Calculer la médiane des nombres de fenêtres
  median_windows = int(np.median(window_counts))
  print(f"Médiane des fenêtres : {median_windows}")

  aligned_modalities = []
  for modality in modalities:
      num_windows = modality.shape[0]
      if num_windows > median_windows:
          # Tronquer si le nombre de fenêtres est supérieur à la médiane
          aligned_modality = modality[:median_windows]
      elif num_windows < median_windows:
          # Compléter si le nombre de fenêtres est inférieur à la médiane
          padding_needed = median_windows - num_windows
          # Répéter les dernières fenêtres pour compléter
          padding = np.tile(modality[-1:], (padding_needed, 1, 1))
          aligned_modality = np.concatenate([modality, padding], axis=0)
      else:
          # Pas de changement si le nombre de fenêtres est égal à la médiane
          aligned_modality = modality
      aligned_modalities.append(aligned_modality)

  return aligned_modalities"""

'def align_data_to_median(modalities):\n\n  # Obtenir le nombre de fenêtres pour chaque modalité\n  window_counts = [modality.shape[0] for modality in modalities]\n  # Calculer la médiane des nombres de fenêtres\n  median_windows = int(np.median(window_counts))\n  print(f"Médiane des fenêtres : {median_windows}")\n\n  aligned_modalities = []\n  for modality in modalities:\n      num_windows = modality.shape[0]\n      if num_windows > median_windows:\n          # Tronquer si le nombre de fenêtres est supérieur à la médiane\n          aligned_modality = modality[:median_windows]\n      elif num_windows < median_windows:\n          # Compléter si le nombre de fenêtres est inférieur à la médiane\n          padding_needed = median_windows - num_windows\n          # Répéter les dernières fenêtres pour compléter\n          padding = np.tile(modality[-1:], (padding_needed, 1, 1))\n          aligned_modality = np.concatenate([modality, padding], axis=0)\n      else:\n          # Pas de changeme

In [ ]:
"""# Données segmentées
modalities = [segmented_dataImu, segmented_dataEmg, segmented_dataIps]

# Aligner les données à la médiane
aligned_modalities = align_data_to_median(modalities)

# Vérifier les dimensions après alignement
for i, aligned_modality in enumerate(aligned_modalities):
    print(f"Modalité {i + 1} après alignement : {aligned_modality.shape}")

"""

In [ ]:
"""merged_results = []

for participant_id in normalized_dataEmg['id'].unique():
    emg_participant = normalized_dataEmg[normalized_dataEmg['id'] == participant_id]
    imu_participant = normalized_dataImu[normalized_dataImu['id'] == participant_id]
    ips_participant = normalized_dataIps[normalized_dataIps['id'] == participant_id]

    # Fusionner pour le participant actuel
    merged_participant = merge_modalities(emg_participant, imu_participant, ips_participant)

    # Sauvegarder les résultats pour éviter de surcharger la RAM
    merged_results.append(merged_participant)

    # Optionnel : Sauvegarder temporairement sur disque pour libérer de la mémoire
    merged_participant.to_csv(f"merged_participant_{participant_id}.csv", index=False)
"""

In [ ]:
"""1 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
...
25 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> still -> ips_1.csv , emg_1.csv, imu_1.csv"""
